**Sample Forex Factory webscraper**

1. Install and import dependencies

In [1]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install chromedriver-binary

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [6]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import chromedriver_binary
import pymysql

2. Create a browser instance

In [7]:
#Creating a webdriver instance
driver = webdriver.Edge()
#driver = webdriver.Chrome()         ... in case you have google chrome installed

3. Use browser instance to access web pages

In [8]:
# Opening website
driver.get("https://www.forexfactory.com")

4. Extract HTML from web page

In [9]:
#Extracting data from website
src = driver.page_source
# Now using beautiful soup
mysoup = BeautifulSoup(src, 'html')

5. Extract specific HTML element

In [10]:
#Getting the 'today' section
today = mysoup.find("tbody")

6. List all upcoming events

In [11]:
#If a news item has a red marker, we will add it to our array of important events.

all_events = today.find_all("tr", {"class": "calendar__row"})
#newsitems = username_element.get_text()

7. Filter out important events only

In [12]:
coming_soon = []  # List of all events that are coming soon
imp_events = []  # List of important events
high_imp = []

past_Events_Classname = 'calendar__row--grey'
new_day_Classname = 'calendar__row--new-day'

#if an event is not a past event, add it to the list
for i in all_events:
    if past_Events_Classname not in i['class']:
        coming_soon.append(i)

#filter out important events
for j in coming_soon:
    #print("Coming soon: ", j, "\n")
    high_imp.append(j.find("span", {"title": "Medium Impact Expected"}))

for p in high_imp:
    if p != None:
        print("High: ", p.parent.parent)
    else:
        high_imp.remove(p)
    
for m in high_imp:
    if m != None:
        single_item = m.parent.parent
        print("This was high: \n", single_item.prettify())
        imp_events.append(single_item)
    #imp_events.append(high_imp.parent())
    #print(high_imp)
    #if high_imp != None:
    #    imp_events.append(j)
    #else:
    #    pass

for k in imp_events:
    print("Important: ", k, "\n")

This was high: 
 <tr class="calendar__row calendar__row--no-grid" data-event-id="130022" data-touchable="">
 <td class="calendar__cell calendar__date">
  <!-- -->
 </td>
 <td class="calendar__cell calendar__time">
  <!-- -->
 </td>
 <td class="calendar__cell calendar__sub crunched nowrap">
  <div>
   <a class="mini-sub mini-sub" href="/login" title="Subscribe to ISM Manufacturing Prices">
    <span class="icon icon--mini-unsubscribed">
    </span>
   </a>
   <!-- -->
   <!-- -->
  </div>
 </td>
 <td class="calendar__cell calendar__currency">
  <span class="">
   USD
  </span>
 </td>
 <td class="calendar__cell calendar__impact">
  <span class="icon icon--ff-impact-ora" title="Medium Impact Expected">
  </span>
  <div class="calendar__impact-icon calendar__impact-icon--print">
   <img alt="" height="12" src="https://resources.faireconomy.media/images/sprites/ff-impact-ora.png" width="14"/>
  </div>
 </td>
 <td class="calendar__cell calendar__event event" colspan="1">
  <div>
   <span cla

8. Package the details before sending to a sql database

In [13]:
#Empty list that will hold all the news items
my_posts = []

#Populate the list with the news items
for n in imp_events:
    time = n.find("td", {"class": "calendar__time"})
    if time.div is not None:
        time = time.div.span.find_next_sibling().text
    else:
        time = "N/A"                                                                        #sorted
    currency = n.find("td", {"class": "calendar__cell calendar__currency"}).span.text       #sorted
    title = n.find("span", {"class": "calendar__event-title"}).text                         #sorted
    previous = n.find("td", {"class": "calendar__previous"}).span.text                      #sorted
    forecast = n.find("td", {"class": "calendar__forecast"}).span.text                      #sorted
    
    my_posts.append({"time": time, "currency": currency, "title": title, "previous": previous, "forecast": forecast})

for mp in my_posts:
    print("\n Post: \n", mp)



 Post: 
 {'time': 'N/A', 'currency': 'USD', 'title': 'ISM Manufacturing Prices', 'previous': '48.4', 'forecast': '48.9'}

 Post: 
 {'time': 'N/A', 'currency': 'USD', 'title': 'FOMC Member Harker Speaks', 'previous': '', 'forecast': ''}


9. Connect to a sql database

In [57]:
#Start a connection to the server
conn = pymysql.connect(host='127.0.0.1', user='fxfactory_scraper', passwd='&Asdf1234', db='scraping', autocommit=True)

cur = conn.cursor()
cur.execute("select @@version")
output = cur.fetchall()

# To close the connection
#conn.close()

#Select the right db
cur.execute("USE scraping")

#Modify the db to accept unicode
cur.execute("ALTER DATABASE scraping CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci;")
cur.execute("ALTER TABLE upcoming CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
cur.execute("ALTER TABLE upcoming CHANGE title title VARCHAR(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")

cur.execute("ALTER DATABASE scraping CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci;")
cur.execute("ALTER TABLE alltime CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
cur.execute("ALTER TABLE alltime CHANGE title title VARCHAR(200) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")

0

10. Send the news to a SQL database

In [62]:
#Switch to the right db
#import time

#Create tables
#cur.execute("CREATE TABLE upcoming (id BIGINT(7) NOT NULL AUTO_INCREMENT, time VARCHAR(200), currency VARCHAR(200), title VARCHAR(200), previous VARCHAR(200), forecast VARCHAR(200), created TIMESTAMP DEFAULT CURRENT_TIMESTAMP, PRIMARY KEY(id))")
#cur.execute("CREATE TABLE alltime (id BIGINT(7) NOT NULL AUTO_INCREMENT, time VARCHAR(200), currency VARCHAR(200), title VARCHAR(200), previous VARCHAR(200), forecast VARCHAR(200), created TIMESTAMP DEFAULT CURRENT_TIMESTAMP, PRIMARY KEY(id))")
print("Tables created:")
#print(cur.execute("DESCRIBE alltime;"))
#print(cur.execute("SHOW TABLES;"))
#cur.execute("SELECT * FROM alltime;")
print("Done")

#Send data to tables
#cur.execute("INSERT INTO alltime (time, currency, title, previous, forecast) VALUES (adf, `adf`, `sadfa`, `adsfad`, `asdfasdfa`)")

for post in my_posts:
    print("My post is: ", post)
    cur.execute("INSERT INTO upcoming (time, currency, title, previous, forecast) VALUES (%s, %s, %s, %s, %s)", (post['time'], post["currency"], post["title"], post["previous"], post["forecast"]))

#for post in my_posts:
#cur.execute("INSERT INTO upcoming (time, currency, title, previous, forecast) VALUES (today, `CNY`, `Manufacturing PMI`, `11.3`, `12.9`);")

cur.execute("SELECT * FROM upcoming")


Tables created:
Done
My post is:  {'time': 'N/A', 'currency': 'USD', 'title': 'ISM Manufacturing Prices', 'previous': '48.4', 'forecast': '48.9'}
My post is:  {'time': 'N/A', 'currency': 'USD', 'title': 'FOMC Member Harker Speaks', 'previous': '', 'forecast': ''}


2